# Breakdown DEM
This notebook uses the clipped DEM to derive topographical statistics per plot.

In [1]:
import os
os.chdir("..")

We want to open the DEM file in meters of elevation, so we can calculate statistics.

In [2]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling

dem_path = 'data/DEM/RegressionRidge_DEM.tif'
dem_utm_path = 'data/DEM/RegressionRidge_DEM_utm.tif'  

# Reproject DEM to UTM (meters)
with rasterio.open(dem_path) as src:
    if src.crs.to_epsg() != 32610:  #
        dst_crs = 'EPSG:32610'
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds
        )
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })
        
        
        # save a copy of projection for later loading
        with rasterio.open(dem_utm_path, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.bilinear
                )
        dem_file = dem_utm_path
    else:
        dem_file = dem_path

In [3]:
# Load DEM
with rasterio.open(dem_file) as src:
    dem = src.read(1)   
    transform = src.transform
    nodata = src.nodata
    xres = transform.a
    yres = transform.e

In [4]:
import numpy as np

# mask nodata
dem = np.where(dem == nodata, np.nan, dem)

In [5]:
# compute slope and aspect
dy, dx = np.gradient(dem, abs(yres), abs(xres))

dxx, dxy = np.gradient(dx, abs(yres), abs(xres))
dyx, dyy = np.gradient(dy, abs(yres), abs(xres)) 

curvature = dxx + dyy 
profile_curv = (
    (dxx * dx**2 + 2 * dxy * dx * dy + dyy * dy**2)
    / ((dx**2 + dy**2) * (1 + dx**2 + dy**2)**1.5 + 1e-9)
)

plan_curv = (
    (dxx * dy**2 - 2 * dxy * dx * dy + dyy * dx**2)
    / ((dx**2 + dy**2) * (1 + dx**2 + dy**2)**0.5 + 1e-9)
)

# slope in degrees
slope_deg = np.degrees(np.arctan(np.sqrt(dx**2 + dy**2)))

# aspect in degrees, 0 = north
aspect_rad = np.arctan2(-dx, dy)
aspect_deg = (np.degrees(aspect_rad) + 360) % 360

# elevation deviation
elev_dev = dem - np.nanmean(dem)

/home/simonhans/anaconda3/envs/GrapeExpectations/lib/python3.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in remainder


Now that we have these statistics calculated for our dem, we can apply our polygons and get plot-wide statistics.

First load polygons. Then use rasterstats zonal stats applied to different vineyard-wide computed topographies.

In [6]:
import geopandas as gpd

vineyard_path = 'data/polygons/RegressionRidge.geojson'
plots_path = 'data/polygons/RegressionRidge_smol.geojson'

# Load vineyard polygons
vineyard = gpd.read_file(vineyard_path)
vineyard = vineyard.to_crs(crs=rasterio.open(dem_file).crs)
# vineyard.plot()

# Load plot polygons
plots = gpd.read_file(plots_path)
plots = plots.to_crs(crs=rasterio.open(dem_file).crs)
# plots.plot()

In [7]:
from rasterstats import zonal_stats
import pandas as pd


def calculate_elevation_statistics(gdf):
    # Compute zonal stats
    slope_stats = zonal_stats(gdf, slope_deg, affine=transform, nodata=np.nan,
                              stats=['mean', 'min', 'max'])
    aspect_stats = zonal_stats(gdf, aspect_deg, affine=transform, nodata=np.nan,
                               stats=['mean', 'min', 'max'])
    
    elev_stats = zonal_stats(gdf, dem, affine=transform, nodata=np.nan,
                             stats=['mean', 'min', 'max'])
    
    elev_dev_stats = zonal_stats(gdf, elev_dev, affine=transform, nodata=np.nan,
                             stats=['mean', 'min', 'max'])
    
    curv_stats = zonal_stats(gdf, curvature, affine=transform, nodata=np.nan,
                         stats=['mean', 'min', 'max'])
    
    profile_curv_stats = zonal_stats(gdf, profile_curv, affine=transform, nodata=np.nan,
                         stats=['mean', 'min', 'max'])

    
    plan_curv_stats = curv_stats = zonal_stats(gdf, plan_curv, affine=transform, nodata=np.nan,
                         stats=['mean', 'min', 'max'])


    # Combine into dataframe
    df_features = pd.DataFrame({
        'plot_id': gdf.index,
        'slope_mean': [s['mean'] for s in slope_stats],
        'slope_min': [s['min'] for s in slope_stats],
        'slope_max': [s['max'] for s in slope_stats],
        
        'curve_mean': [s['mean'] for s in curv_stats],
        'curve_min': [s['min'] for s in curv_stats],
        'curve_max': [s['max'] for s in curv_stats],
        
        'pro_curve_mean': [s['mean'] for s in profile_curv_stats],
        'pro_curve_min': [s['min'] for s in profile_curv_stats],
        'pro_curve_max': [s['max'] for s in profile_curv_stats],
        
        'plan_curve_mean': [s['mean'] for s in plan_curv_stats],
        'plan_curve_min': [s['min'] for s in plan_curv_stats],
        'plan_curve_max': [s['max'] for s in plan_curv_stats],
        
        
        'aspect_mean': [s['mean'] for s in aspect_stats],
        'aspect_min': [s['min'] for s in aspect_stats],
        'aspect_max': [s['max'] for s in aspect_stats],
        'elev_min': [s['min'] for s in elev_stats],
        'elev_max': [s['max'] for s in elev_stats],
        'elev_mean': [s['mean'] for s in elev_stats],
        'elev_dev_min': [s['min'] for s in elev_dev_stats],
        'elev_dev_max': [s['max'] for s in elev_dev_stats],
        'elev_dev_mean': [s['mean'] for s in elev_dev_stats]
    })

    df_features = df_features.join(gdf)
    df_features['total_relief'] = df_features['elev_max'] - df_features['elev_min']

    gdf_features = gpd.GeoDataFrame(df_features, geometry=gdf.geometry, crs=gdf.crs)
    return gdf_features

# calculate vineyard-wide statistics
vineyard_features = calculate_elevation_statistics(vineyard)

# calculate plot-level statistics
plot_features = calculate_elevation_statistics(plots)

# set vineyard plot id to append to full df
vineyard_features.loc[0, 'plot_id'] = 'full'

df_features = pd.concat([plot_features, vineyard_features])
gdf_features = gpd.GeoDataFrame(
    df_features,
    geometry=df_features['geometry'], 
    crs=rasterio.open(dem_file).crs
)


In [8]:
# compute area in square meters
gdf_features['area_m2'] = gdf_features.geometry.area

# convert to hectares -- commonly used in agg
gdf_features['area_ha'] = gdf_features['area_m2'] / 10000

In [9]:
# select pot features. index 65 is teh full vineyard
plot_features = gdf_features.reset_index().loc[0:64]

Now to visualize our work!

In [10]:
plot_features

,index,plot_id,slope_mean,slope_min,slope_max,curve_mean,curve_min,curve_max,pro_curve_mean,pro_curve_min,...,elev_min,elev_max,elev_mean,elev_dev_min,elev_dev_max,elev_dev_mean,geometry,total_relief,area_m2,area_ha
0,0,0,7.116202,1.671869,12.657069,0.000475,-0.042193,0.036974,-0.000320,-0.043223,...,202.678589,227.649673,216.987355,17.714752,42.685837,32.023511,"POLYGON ((749515.552 5085017.582, 749593.102 5...",24.971085,37742.453705,3.774245
1,1,1,7.157215,2.141364,15.119311,-0.000136,-0.058244,0.040260,0.000042,-0.033721,...,99.260628,120.873428,110.290033,-85.703209,-64.090408,-74.673799,"POLYGON ((749293.527 5084266.311, 749284.600 5...",21.612801,31915.687817,3.191569
2,2,2,11.803730,5.291763,19.373894,0.000031,-0.034337,0.036800,0.000070,-0.034964,...,105.712486,124.070328,115.284763,-79.251350,-60.893509,-69.679062,"POLYGON ((749683.152 5084433.659, 749663.238 5...",18.357841,10407.243140,1.040724
3,3,3,9.123407,1.320997,14.460165,0.000095,-0.031373,0.025294,0.000140,-0.028785,...,95.084496,109.766037,102.207162,-89.879341,-75.197800,-82.756664,"POLYGON ((749671.796 5084356.545, 749653.826 5...",14.681541,12707.534383,1.270753
4,4,4,6.518723,0.240493,15.043138,0.000038,-0.061499,0.064917,0.000613,-0.062695,...,130.967911,178.674332,152.926721,-53.995926,-6.289505,-32.037104,"POLYGON ((750026.827 5084777.385, 750031.798 5...",47.706421,127044.721505,12.704472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,60,60,4.760527,0.158230,13.548397,-0.000008,-0.050380,0.058834,-0.000153,-0.046366,...,230.046875,255.310486,243.288999,45.083038,70.346649,58.325193,"POLYGON ((749906.742 5085494.977, 749923.026 5...",25.263611,94504.390271,9.450439
61,61,61,3.481692,0.022498,13.487131,-0.000261,-0.040569,0.043431,-0.000353,-0.055958,...,246.021927,261.757812,254.108077,61.058090,76.793976,69.144233,"POLYGON ((750175.532 5085581.273, 750081.328 5...",15.735886,46240.992006,4.624099
62,62,62,3.963347,0.045724,8.905317,-0.000188,-0.043597,0.056039,-0.000059,-0.045036,...,230.808212,253.110870,243.790732,45.844376,68.147034,58.826921,"POLYGON ((749819.022 5085646.705, 749476.313 5...",22.302658,96198.910650,9.619891
63,63,63,3.339648,0.010571,10.322133,-0.000130,-0.065293,0.073079,-0.000182,-0.057799,...,218.849838,241.400696,231.685729,33.886002,56.436859,46.721879,"POLYGON ((749602.788 5085360.138, 749606.595 5...",22.550858,97439.734996,9.743973


In [11]:
import matplotlib.pyplot as plt

with rasterio.open(dem_file) as src:
    dem = src.read(1, masked=True)
    raster_crs = src.crs
    extent = [src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top]

fig, ax = plt.subplots(figsize=(10, 6))

# DEM
im = ax.imshow(
    dem,
    extent=extent,
    origin='upper',
    cmap='terrain',
    vmin=25,
    vmax=350
)

# create a new axes for the left colorbar [left, bottom, width, height]
cbar_ax_left = fig.add_axes([0.03, 0.15, 0.02, 0.7])  # tweak numbers as needed
cbar_dem = fig.colorbar(im, cax=cbar_ax_left)
cbar_dem.set_label('Elevation (m)', fontsize=15)
cbar_dem.ax.tick_params(labelsize=12)

# vineyard
vineyard.plot(ax=ax, facecolor='none', edgecolor='k', linewidth=2)

# plot features without legend
plot_gdf = plot_features.copy()
plot_gdf.plot(
    ax=ax,
    column='pro_curve_mean',
    cmap='viridis',
    legend=False,
    alpha=0.8,
    edgecolor='black',
    linewidth=0.5
)

# slope colorbar on the right (default)
sm = plt.cm.ScalarMappable(
    cmap='viridis',
    norm=plt.Normalize(vmin=plot_features['curve_max'].min(),
                       vmax=plot_features['curve_max'].max())
)
sm._A = []
cbar_aspect = fig.colorbar(sm, ax=ax, fraction=0.035, pad=0.04)
cbar_aspect.set_label('Max Slope (degrees)', fontsize=15)
cbar_aspect.ax.tick_params(labelsize=12)

fig.suptitle('Regression Ridge\nClipped DEM with Max Slope', fontsize = 18)
ax.set_xlabel('Longitude', fontsize = 15)
ax.set_ylabel('Latitude', fontsize = 15)
ax.set_xticks([])
ax.set_yticks([])
plt.savefig('img/dem_w_slope.png')
plt.show()


Save our plot topography features to a pickle for later use.

In [12]:
gdf_features.to_pickle('data/plot_elev_features.pkl')